In [28]:
!pip install kagglehub[pandas-datasets]

In [13]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

# Define o nome EXATO do arquivo que existe dentro do dataset no Kaggle
file_path = "pip.csv"

# Carrega o dataset diretamente do Kaggle
df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS,
                            "isaaclopgu/share-of-population-living-in-extreme-poverty",
                            file_path,)

print("Primeiros 5 registros:")
display(df.head())

/tmp/ipython-input-477292937.py:8: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'share-of-population-living-in-extreme-poverty' dataset.
Primeiros 5 registros:


,region_name,region_code,country_name,country_code,reporting_year,reporting_level,survey_acronym,survey_coverage,survey_year,welfare_type,...,reporting_pop,reporting_gdp,reporting_pce,is_interpolated,distribution_type,estimation_type,spl,spr,pg,estimate_type
0,Sub-Saharan Africa,SSA,Angola,AGO,2000,national,HBS,national,2000.21,consumption,...,16310860.19,1932.988479,NaN,False,micro,survey,3.950,0.371469,9.609202,NaN
1,Sub-Saharan Africa,SSA,Angola,AGO,2008,national,IBEP-MICS,national,2008.50,consumption,...,21996714.00,3193.287723,1268.614278,False,micro,survey,3.967,0.353358,6.838756,NaN
2,Sub-Saharan Africa,SSA,Angola,AGO,2018,national,IDREA,national,2018.17,consumption,...,31480496.09,2775.746423,1605.110432,False,micro,survey,3.240,0.421619,10.964242,NaN
3,Europe & Central Asia,ECA,Albania,ALB,1996,national,EWS,national,1996.00,consumption,...,3168033.00,1633.552000,1716.336801,False,micro,survey,5.010,0.203902,4.209428,NaN
4,Europe & Central Asia,ECA,Albania,ALB,2002,national,LSMS,national,2002.00,consumption,...,3051010.00,2247.498080,1685.203078,False,micro,survey,4.859,0.224404,4.429310,NaN


In [15]:
# Criar dimensão País/Região
dim_country = df[['region_name', 'region_code', 'country_name', 'country_code']].drop_duplicates().reset_index(drop=True)
dim_country['country_id'] = dim_country.index + 1
dim_country = dim_country[['country_id', 'region_name', 'region_code', 'country_name', 'country_code']]

print("Dimensão País/Região:")
print(dim_country.head())

Dimensão País/Região:
   country_id                  region_name region_code          country_name  \
0           1           Sub-Saharan Africa         SSA                Angola   
1           2        Europe & Central Asia         ECA               Albania   
2           3  Other High Income Countries         OHI  United Arab Emirates   
3           4    Latin America & Caribbean         LAC             Argentina   
4           5        Europe & Central Asia         ECA               Armenia   

  country_code  
0          AGO  
1          ALB  
2          ARE  
3          ARG  
4          ARM  


In [23]:
# Dimensão Ano
dim_year = df[['reporting_year', 'reporting_level', 'survey_year', 'survey_acronym', 'survey_coverage']].drop_duplicates().reset_index(drop=True)
dim_year['year_id'] = dim_year.index + 1
dim_year = dim_year[['year_id', 'reporting_year', 'reporting_level', 'survey_year', 'survey_acronym', 'survey_coverage']]

print("Dimensão Ano / Reportagem:")
print(dim_year.head())

Dimensão Ano / Reportagem:
   year_id  reporting_year reporting_level  survey_year survey_acronym  \
0        1            2000        national      2000.21            HBS   
1        2            2008        national      2008.50      IBEP-MICS   
2        3            2018        national      2018.17          IDREA   
3        4            1996        national      1996.00            EWS   
4        5            2002        national      2002.00           LSMS   

  survey_coverage  
0        national  
1        national  
2        national  
3        national  
4        national  


In [24]:
# Dimensão Welfare / Linha de pobreza
dim_welfare = df[['welfare_type', 'poverty_line', 'distribution_type', 'estimation_type', 'estimate_type']].drop_duplicates().reset_index(drop=True)
dim_welfare['welfare_id'] = dim_welfare.index + 1
dim_welfare = dim_welfare[['welfare_id', 'welfare_type', 'poverty_line', 'distribution_type', 'estimation_type', 'estimate_type']]

print("Dimensão Welfare / Linha de pobreza:")
print(dim_welfare.head())

Dimensão Welfare / Linha de pobreza:
   welfare_id welfare_type  poverty_line distribution_type estimation_type  \
0           1  consumption            10             micro          survey   
1           2       income            10             micro          survey   
2           3       income            10             group          survey   
3           4  consumption            10             group          survey   
4           5       income            10         synthetic          survey   

   estimate_type  
0            NaN  
1            NaN  
2            NaN  
3            NaN  
4            NaN  


In [25]:
# Dimensão Comparabilidade
dim_comparability = df[['survey_comparability', 'comparable_spell']].drop_duplicates().reset_index(drop=True)
dim_comparability['comparability_id'] = dim_comparability.index + 1
dim_comparability = dim_comparability[['comparability_id', 'survey_comparability', 'comparable_spell']]

print("Dimensão Comparabilidade:")
print(dim_comparability.head())

Dimensão Comparabilidade:
   comparability_id  survey_comparability comparable_spell
0                 1                     0             2000
1                 2                     1             2008
2                 3                     2             2018
3                 4                     0             1996
4                 5                     1      2002 - 2012


In [26]:
# Tabela Fato Pobreza
fact_poverty = df.merge(dim_country, on=['country_name', 'country_code', 'region_name', 'region_code']) \
                 .merge(dim_year, on='reporting_year') \
                 .merge(dim_welfare, on=['welfare_type','poverty_line','distribution_type','estimation_type','estimate_type']) \
                 .merge(dim_comparability, on=['survey_comparability','comparable_spell'])

fact_poverty = fact_poverty[['country_id', 'year_id', 'welfare_id', 'comparability_id',
                             'headcount','poverty_gap','poverty_severity','watts','mean','median','gini']]


In [27]:
print("Dimensão País/Região:")
print(dim_country.head())
print("\nDimensão Ano / Reportagem:")
print(dim_year.head())
print("\nDimensão Bem-estar / Linha de pobreza:")
print(dim_welfare.head())
print("\nFato Pobreza:")
print(fact_poverty.head())

Dimensão País/Região:
   country_id                  region_name region_code          country_name  \
0           1           Sub-Saharan Africa         SSA                Angola   
1           2        Europe & Central Asia         ECA               Albania   
2           3  Other High Income Countries         OHI  United Arab Emirates   
3           4    Latin America & Caribbean         LAC             Argentina   
4           5        Europe & Central Asia         ECA               Armenia   

  country_code  
0          AGO  
1          ALB  
2          ARE  
3          ARG  
4          ARM  

Dimensão Ano / Reportagem:
   year_id  reporting_year reporting_level  survey_year survey_acronym  \
0        1            2000        national      2000.21            HBS   
1        2            2008        national      2008.50      IBEP-MICS   
2        3            2018        national      2018.17          IDREA   
3        4            1996        national      1996.00            EWS 

In [ ]:
# 5. Gravar tudo no datamart.db
# -----------------------------
def salvar_datamart(df_dim_country, df_dim_year, df_dim_welfare, df_dim_comparability, df_fact_poverty):
    conn_dm = sqlite3.connect(DATAMART_DB)

    # Salva dados no data mart, substituindo se as tabelas já existirem
    df_dim_country.to_sql("dim_country", conn_dm, if_exists="replace", index=False)
    df_dim_year.to_sql("dim_year", conn_dm, if_exists="replace", index=False)
    df_dim_welfare.to_sql("dim_welfare", conn_dm, if_exists="replace", index=False)
    df_dim_comparability.to_sql("dim_comparability", conn_dm, if_exists="replace", index=False)
    df_fact_poverty.to_sql("fact_poverty", conn_dm, if_exists="replace", index=False)

    conn_dm.close()

In [ ]:
# Importar sqlite3
import sqlite3

# Definir o nome do banco de dados do datamart
DATAMART_DB = "/content/drive/MyDrive/POSGRADUACAO/datamart.db"

print("Salvando no datamart...")
salvar_datamart(dim_country, dim_year, dim_welfare, dim_comparability, fact_poverty)
print(f"Datamart gerado em {DATAMART_DB}")

Salvando no datamart...
Datamart gerado em /content/drive/MyDrive/POSGRADUACAO/datamart.db
